| | |
|-|-|
| __Use case__ | The user should be able to combine filters using OR and AND operations. |
| __Idea__ | Using sets operations on the filtering results. |
| __Problem__ | The combination of filters can be very complex and should be simplified. |
| __Solution__ | Converting the logic expression to Conjunctive Normal Form (CNF) and then applying sets operations. |

### Simplifying expressions

In [1]:
from sympy.logic.boolalg import And, Or, Not, Implies, Equivalent
from sympy.logic.boolalg import to_cnf
from sympy.abc import A, E, F

a) (E ∧ A ∧ F ) ∨ ( A ∧ F )

In [2]:
to_cnf((E & A & F ) | ( A & F ),True)

A & F

b) (A ∧ E) V (F ∧ E V A) ∧ (A V F)

In [3]:
to_cnf((A & E) | (F & E | A) & (A | F),True)

(A | E) & (A | F)

c) ((A ∨ F) ∨ E ∧ (A ∨ E) ∧ (F ∨ E)) ∧ (A ∧ E ∨ F ∧ (E ∨ A))

In [4]:
to_cnf(((A | F) | E & (A | E) & (F | E)) & (A & E | F & (E | A)),True)

(A | E) & (A | F) & (E | F)

### Example 1 : Simple sets of numbers

In [5]:
import random

def sort_set(x):
    x = list(x)
    x.sort()
    return x

a) (E ∧ A ∧ F ) ∨ ( A ∧ F ) = A ∧ F

In [6]:
E = { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
A = { 1, 6, 7, 8, 9, 10, 11, 12, 13}
F = { 1, 2, 3, 4, 5, 6, 0, 15, 14}

In [7]:
E = { random.randint(1,50) for i in range(35)}
A = { random.randint(1,50) for i in range(35)}
F = { random.randint(1,50) for i in range(35)}

In [8]:
EAF = (E.intersection(A)).intersection(F)
AF = A.intersection(F)
EAF_or_AF = EAF.union(AF)

In [9]:
print(sort_set(AF))
print(sort_set(EAF_or_AF))

[1, 6, 11, 22, 23, 26, 32, 36, 43, 48, 49]
[1, 6, 11, 22, 23, 26, 32, 36, 43, 48, 49]


EAF_or_AF == AF

b) (A ∧ E) V (F ∧ E V A) ∧ (A V F) =  (A ∨ E) ∧ (A ∨ F)

In [10]:
AE = A.intersection(E)
FEorA = (F.intersection(E)).union(A)
AorF = A.union(F)
AE_or_FEorA_and_AorF = (AE.union(FEorA)).intersection(AorF)

In [11]:
AorE = A.union(E)
AorF = A.union(F)
AorE_and_AorF = AorE.intersection(AorF)

In [12]:
print(sort_set(AE_or_FEorA_and_AorF))
print(sort_set(AorE_and_AorF))

[1, 2, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 19, 22, 23, 25, 26, 28, 29, 30, 32, 33, 36, 38, 39, 41, 43, 44, 45, 47, 48, 49, 50]
[1, 2, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 19, 22, 23, 25, 26, 28, 29, 30, 32, 33, 36, 38, 39, 41, 43, 44, 45, 47, 48, 49, 50]


AE_or_FEorA_and_AorF == AorE_and_AorF

c) ((A ∨ F) ∨ E ∧ (A ∨ E) ∧ (F ∨ E)) ∧ (A ∧ E ∨ F ∧ (E ∨ A)) = (A ∨ E) ∧ (A ∨ F) ∧ (E ∨ F)

In [13]:
left = ((((A.union(F)).union(E)).intersection(A.union(E))).intersection(F.union(E))).intersection(((A.intersection(E)).union(F)).intersection(E.union(A)))
right = ((A.union(E)).intersection(A.union(F))).intersection(E.union(F))
print(sort_set(left))
print(sort_set(right))

[1, 2, 6, 7, 10, 11, 16, 19, 22, 23, 25, 26, 28, 29, 30, 32, 33, 36, 39, 41, 43, 45, 47, 48, 49, 50]
[1, 2, 6, 7, 10, 11, 16, 19, 22, 23, 25, 26, 28, 29, 30, 32, 33, 36, 39, 41, 43, 45, 47, 48, 49, 50]


left == right

### Example 2 : LTL filters

In [14]:
import pandas as pd
import pm4py
import pm4py.algo.filtering.pandas.ltl as ltl
import os

# get data from csv file
df = pd.read_csv(os.path.join("logs", "detail_incident_activity.csv"))
# dataframe formatting
df = pm4py.format_dataframe(df, case_id="Incident ID", activity_key="IncidentActivity_Type", timestamp_key="DateStamp")

# KeyError: "['org:resource'] not in index"
# org:resource = who performed a given activity
df.rename(columns={'Assignment Group':'org:resource'}, inplace=True) 


# drop unnecessary columns
df.drop(['Incident ID', 'DateStamp', 'IncidentActivity_Number','IncidentActivity_Type', 'KM number','Interaction ID','@@index'], inplace=True, axis=1)
# convert dataframe to logs format
logs = pm4py.convert_to_event_log(df)

F = ltl.ltl_checker.four_eyes_principle(df, "Open", "Closed")
E = ltl.ltl_checker.A_eventually_B(df, "Open", "Update")
A = ltl.ltl_checker.attr_value_different_persons(df, "Update")

F= F.drop_duplicates(list(F.columns))
E = E.drop_duplicates(list(E.columns))
A = A.drop_duplicates(list(A.columns))

In [15]:
def df_intersection(A, B):
    return pd.merge(A, B, how ='inner', on =list(A.columns)).reset_index().drop(columns=["index"])

def df_union(A,B):
    return pd.concat([A,B]).drop_duplicates(list(A.columns)).reset_index().drop(columns=["index"])

a) (E ∧ A ∧ F ) ∨ ( A ∧ F ) = A ∧ F

In [16]:
left = df_union(df_intersection(df_intersection(E,A),F),df_intersection(A,F))
left.head(5)

,org:resource,case:concept:name,concept:name,time:timestamp
0,TEAM9999,IM0000038,Open,2013-01-15 14:36:33+00:00
1,TEAM0003,IM0000038,Operator Update,2013-01-15 15:06:12+00:00
2,TEAM0003,IM0000038,Assignment,2013-01-15 15:06:12+00:00
3,TEAM0003,IM0000038,Operator Update,2013-01-15 16:03:24+00:00
4,TEAM0003,IM0000038,Communication with customer,2013-01-15 16:17:52+00:00


In [17]:
right = df_intersection(A,F)
right.head(5)

,org:resource,case:concept:name,concept:name,time:timestamp
0,TEAM9999,IM0000038,Open,2013-01-15 14:36:33+00:00
1,TEAM0003,IM0000038,Operator Update,2013-01-15 15:06:12+00:00
2,TEAM0003,IM0000038,Assignment,2013-01-15 15:06:12+00:00
3,TEAM0003,IM0000038,Operator Update,2013-01-15 16:03:24+00:00
4,TEAM0003,IM0000038,Communication with customer,2013-01-15 16:17:52+00:00


In [18]:
print(len(left.index))
print(len(right.index))

57236
57236


left == right

b) (A ∧ E) V (F ∧ E V A) ∧ (A V F) =  (A ∨ E) ∧ (A ∨ F)

In [19]:
left = df_intersection(df_union(df_intersection(A,E),df_union(df_intersection(F,E),A)),df_union(A,F))
left.head(5)

,org:resource,case:concept:name,concept:name,time:timestamp
0,TEAM9999,IM0000038,Open,2013-01-15 14:36:33+00:00
1,TEAM0003,IM0000038,Operator Update,2013-01-15 15:06:12+00:00
2,TEAM0003,IM0000038,Assignment,2013-01-15 15:06:12+00:00
3,TEAM0003,IM0000038,Operator Update,2013-01-15 16:03:24+00:00
4,TEAM0003,IM0000038,Communication with customer,2013-01-15 16:17:52+00:00


In [20]:
right = df_intersection(df_union(A,E),df_union(A,F))
right.head(5)

,org:resource,case:concept:name,concept:name,time:timestamp
0,TEAM0003,IM0000015,Update,2013-01-05 13:58:00+00:00
1,TEAM0003,IM0000015,Update,2013-01-05 14:06:00+00:00
2,TEAM0002,IM0000015,Update from customer,2013-02-13 12:31:12+00:00
3,TEAM0002,IM0000015,Update from customer,2013-02-14 11:07:09+00:00
4,TEAM0002,IM0000015,Assignment,2013-02-14 11:07:09+00:00


In [21]:
print(len(left.index))
print(len(right.index))

191572
191572


left == right

c) ((A ∨ F) ∨ E ∧ (A ∨ E) ∧ (F ∨ E)) ∧ (A ∧ E ∨ F ∧ (E ∨ A)) = (A ∨ E) ∧ (A ∨ F) ∧ (E ∨ F)

In [22]:
left = df_intersection(df_intersection(df_intersection(df_union(df_union(A,F),E),df_union(A,E)),df_union(F,E)),df_intersection(df_union(df_intersection(A,E),F),df_union(E,A)))
left.head(5)

,org:resource,case:concept:name,concept:name,time:timestamp
0,TEAM9999,IM0000038,Open,2013-01-15 14:36:33+00:00
1,TEAM0003,IM0000038,Operator Update,2013-01-15 15:06:12+00:00
2,TEAM0003,IM0000038,Assignment,2013-01-15 15:06:12+00:00
3,TEAM0003,IM0000038,Operator Update,2013-01-15 16:03:24+00:00
4,TEAM0003,IM0000038,Communication with customer,2013-01-15 16:17:52+00:00


In [23]:
right = df_intersection(df_intersection(df_union(A,E),df_union(A,F)),df_union(E,F))
right.head(5)

,org:resource,case:concept:name,concept:name,time:timestamp
0,TEAM9999,IM0000038,Open,2013-01-15 14:36:33+00:00
1,TEAM0003,IM0000038,Operator Update,2013-01-15 15:06:12+00:00
2,TEAM0003,IM0000038,Assignment,2013-01-15 15:06:12+00:00
3,TEAM0003,IM0000038,Operator Update,2013-01-15 16:03:24+00:00
4,TEAM0003,IM0000038,Communication with customer,2013-01-15 16:17:52+00:00


In [24]:
print(len(left.index))
print(len(right.index))

190184
190184


left == right

### Important

#### The same filters with different parameters must be assigned to different variables!

Example:


four_eyes_principle(X,Y) != four_eyes_principle(X,Z)

A = four_eyes_principle(X,Y)

B = four_eyes_principle(X,Z)

four_eyes_principle(X,Y) OR four_eyes_principle(X,Z) = A v B